In [6]:
import pandas as pd
from sales_analysis import AnalysisFactory, profile_strategy

# Cargar el dataset principal
df = pd.read_csv("../../data/sales.csv", parse_dates=["SalesDate"])

# Asegurar que SalesDate sea datetime
df["SalesDate"] = pd.to_datetime(df["SalesDate"])

# Verificar y construir la columna TotalPriceCalculated si no existe
if "TotalPriceCalculated" not in df.columns:
    if "Price" not in df.columns:
        # Cargar precios desde products
        products = pd.read_csv("../../data/products.csv")
        df = df.merge(products[["ProductID", "Price"]], on="ProductID", how="left")

    df["TotalPriceCalculated"] = (df["Quantity"] * df["Price"]) * (1 - df["Discount"])

# Mostrar tipos de datos para confirmación
print("Tipos de columnas:\n", df.dtypes)
print("\nPrimeras filas:\n", df[["SalesDate", "TotalPriceCalculated"]].head())

# Aplicar la estrategia optimizada (vectorizada)
strategy = AnalysisFactory.get_strategy("top5_vectorized")
resultado = strategy.analyze(df)

print("\nResultado estrategia vectorizada:")
print(resultado)

# Comparar tiempos de ejecución
print("\nComparación de tiempos:")
for nombre in ["top5_brute", "top5_vectorized"]:
    strat = AnalysisFactory.get_strategy(nombre)
    tiempo = profile_strategy(strat, df, repeat=3)
    print(f"{nombre}: {tiempo:.4f} segundos")



Tipos de columnas:
 SalesID                          int64
SalesPersonID                    int64
CustomerID                       int64
ProductID                        int64
Quantity                         int64
Discount                       float64
TotalPrice                     float64
SalesDate               datetime64[ns]
TransactionNumber               object
Price                          float64
TotalPriceCalculated           float64
dtype: object

Primeras filas:
                 SalesDate  TotalPriceCalculated
0 2018-02-05 07:38:25.430             309.63590
1 2018-02-02 16:03:31.150             437.82200
2 2018-05-03 19:31:56.880            1896.44160
3 2018-04-07 14:43:55.420            1236.01384
4 2018-02-12 15:37:03.940             719.80200

Resultado estrategia vectorizada:
    StartDate     EndDate    TotalSales      Method
0  2018-03-28  2018-04-01  1.666125e+08  vectorized

Comparación de tiempos:
top5_brute: 2.6010 segundos
top5_vectorized: 2.7489 segundos
